In [1]:
import pandas as pd
import numpy as np
import streamlit as st

In [2]:
brasileirao = pd.read_excel('brasileirao2023.xlsx')
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
375,2023,38,GOI,NaN,AME,NaN
376,2023,38,SAN,NaN,FOR,NaN
377,2023,38,SAO,NaN,FLA,NaN
378,2023,38,VAS,NaN,RED,NaN


In [3]:
brasileirao = brasileirao[brasileirao['Score_m'].notnull()]
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
75,2023,8,COR,2.0,FLU,0.0
76,2023,8,INT,2.0,BAH,0.0
77,2023,8,CAM,1.0,PAL,1.0
78,2023,8,RED,2.0,SAN,0.0


In [4]:
times = brasileirao['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = brasileirao[brasileirao['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = brasileirao[brasileirao['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J': [jog],
        '4-V': [vit],
        '5-E' : [emp],
        '6-D': [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,BOT,8,7,0,1,16.0,6.0,21,10.0
0,PAL,8,4,4,0,17.0,7.0,16,10.0
12,SAO,8,4,3,1,14.0,7.0,15,7.0
6,CAM,8,4,2,2,12.0,7.0,14,5.0
9,GRE,8,4,2,2,12.0,11.0,14,1.0
13,CRU,8,4,1,3,12.0,7.0,13,5.0
8,FLA,8,4,1,3,14.0,10.0,13,4.0
10,FLU,8,4,1,3,12.0,8.0,13,4.0
5,FOR,8,3,4,1,12.0,6.0,13,6.0
4,RED,8,3,4,1,13.0,11.0,13,2.0


In [5]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
0,PAL,8,4,4,0,17.0,7.0,16,10.0
1,AME,8,1,1,6,7.0,20.0,4,-13.0
2,CAP,8,4,0,4,11.0,11.0,12,0.0
3,BOT,8,7,0,1,16.0,6.0,21,10.0
4,RED,8,3,4,1,13.0,11.0,13,2.0
5,FOR,8,3,4,1,12.0,6.0,13,6.0
6,CAM,8,4,2,2,12.0,7.0,14,5.0
7,COR,8,2,2,4,8.0,12.0,8,-4.0
8,FLA,8,4,1,3,14.0,10.0,13,4.0
9,GRE,8,4,2,2,12.0,11.0,14,1.0


In [6]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,8,4,4,0,17.0,7.0,16,10.0,0
1,AME,8,1,1,6,7.0,20.0,4,-13.0,3
2,CAP,8,4,0,4,11.0,11.0,12,0.0,2
3,BOT,8,7,0,1,16.0,6.0,21,10.0,4
4,RED,8,3,4,1,13.0,11.0,13,2.0,2
5,FOR,8,3,4,1,12.0,6.0,13,6.0,0
6,CAM,8,4,2,2,12.0,7.0,14,5.0,2
7,COR,8,2,2,4,8.0,12.0,8,-4.0,1
8,FLA,8,4,1,3,14.0,10.0,13,4.0,2
9,GRE,8,4,2,2,12.0,11.0,14,1.0,2


In [7]:
print(kmeans.labels_)

[0 3 2 4 2 0 2 1 2 2 2 1 0 2 1 1 2 3 1 1]


In [8]:
kmeans.cluster_centers_

array([[ 14.66666667,   3.66666667,   3.66666667,   0.66666667,
          7.66666667],
       [  7.66666667,   2.        ,   1.66666667,   4.33333333,
         -5.33333333],
       [ 12.875     ,   3.75      ,   1.625     ,   2.625     ,
          2.75      ],
       [  3.5       ,   0.5       ,   2.        ,   5.5       ,
        -11.5       ],
       [ 21.        ,   7.        ,   0.        ,   1.        ,
         10.        ]])

In [9]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[14.66666667  3.66666667  3.66666667  0.66666667  7.66666667]
1
[ 7.66666667  2.          1.66666667  4.33333333 -5.33333333]
2
[12.875  3.75   1.625  2.625  2.75 ]
3
[  3.5   0.5   2.    5.5 -11.5]
4
[21.  7.  0.  1. 10.]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,14.666667,3.666667,3.666667,0.666667,7.666667
1,1,7.666667,2.000000,1.666667,4.333333,-5.333333
2,2,12.875000,3.750000,1.625000,2.625000,2.750000
3,3,3.500000,0.500000,2.000000,5.500000,-11.500000
4,4,21.000000,7.000000,0.000000,1.000000,10.000000


In [10]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
4,4,21.000000,7.000000,0.000000,1.000000,10.000000
0,0,14.666667,3.666667,3.666667,0.666667,7.666667
2,2,12.875000,3.750000,1.625000,2.625000,2.750000
1,1,7.666667,2.000000,1.666667,4.333333,-5.333333
3,3,3.500000,0.500000,2.000000,5.500000,-11.500000


In [15]:
df_cluster_sort['grupo'] = ['Titulo','Libertadores','Sul-Americana','limbo','Rebaixamento']
st.table(df_cluster)
df_cluster_sort

NameError: name 'st' is not defined

In [12]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
4,4,Titulo
0,0,Libertadores
2,2,Sul-Americana
1,1,limbo
3,3,Rebaixamento


In [13]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,8,4,4,0,17.0,7.0,16,10.0,0
1,AME,8,1,1,6,7.0,20.0,4,-13.0,3
2,CAP,8,4,0,4,11.0,11.0,12,0.0,2
3,BOT,8,7,0,1,16.0,6.0,21,10.0,4
4,RED,8,3,4,1,13.0,11.0,13,2.0,2
5,FOR,8,3,4,1,12.0,6.0,13,6.0,0
6,CAM,8,4,2,2,12.0,7.0,14,5.0,2
7,COR,8,2,2,4,8.0,12.0,8,-4.0,1
8,FLA,8,4,1,3,14.0,10.0,13,4.0,2
9,GRE,8,4,2,2,12.0,11.0,14,1.0,2


In [14]:
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')
# tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
3,BOT,8,7,0,1,16.000000,6.000000,21,10.000000,4
0,PAL,8,4,4,0,17.000000,7.000000,16,10.000000,0
12,SAO,8,4,3,1,14.000000,7.000000,15,7.000000,0
6,CAM,8,4,2,2,12.000000,7.000000,14,5.000000,2
9,GRE,8,4,2,2,12.000000,11.000000,14,1.000000,2
13,CRU,8,4,1,3,12.000000,7.000000,13,5.000000,2
8,FLA,8,4,1,3,14.000000,10.000000,13,4.000000,2
10,FLU,8,4,1,3,12.000000,8.000000,13,4.000000,2
5,FOR,8,3,4,1,12.000000,6.000000,13,6.000000,0
4,RED,8,3,4,1,13.000000,11.000000,13,2.000000,2
